In [1]:
# Loading the data

In [2]:
import pandas as pd
df_graph = pd.read_csv('df_graph_same_docs_as_manual.tsv',sep='\t')

In [3]:
df_graph.head()

,person1,person2,document_ids,weight,wikidata_id1,wikidata_id2,wikidata_description1,wikidata_description2,main_country1,main_country2
0,Harvey Littleton,Wilke Adolfsson,[1248922],1,Q15488059,Q5543008,"Artist, educator",Swedish artist,United States of America,Sweden
1,Durk Valkema,Harvey Littleton,"[1248922, 1265626]",2,Q29422094,Q15488059,Dutch sculptor,"Artist, educator",Kingdom of the Netherlands,United States of America
2,Harvey Littleton,Sybren Valkema,"[1248922, 1266713, 1268564, 1268576, 1268586, ...",11,Q15488059,Q2618110,"Artist, educator",Dutch glass artist (1916-1996),United States of America,Kingdom of the Netherlands
3,Gary Beecham,Harvey Littleton,"[1248922, 1290868]",2,Q5524683,Q15488059,American artist,"Artist, educator",United States of America,United States of America
4,Durk Valkema,Wilke Adolfsson,"[1248922, 1468401, 1266688]",3,Q29422094,Q5543008,Dutch sculptor,Swedish artist,Kingdom of the Netherlands,Sweden


In [4]:
all_countries = set(df_graph['main_country1'].tolist()+df_graph['main_country1'].tolist())
all_countries

{'Austria',
 'Austria-Hungary',
 'Belgium',
 'Canada',
 'Denmark',
 'France',
 'Germany',
 'Italy',
 'Japan',
 'Kingdom of the Netherlands',
 'Norway',
 'Soviet Union',
 'Sweden',
 'Switzerland',
 'United Kingdom',
 'United States of America',
 nan}

In [5]:
from tqdm.auto import tqdm
country_by_person = {}
for _,row in tqdm(df_graph.iterrows()):
    country_by_person[row['person1']] =row['main_country1']
    country_by_person[row['person2']] =row['main_country2']

0it [00:00, ?it/s]

In [6]:
import pickle
pickle.dump(country_by_person,open('country_per_person.p','wb'))

In [5]:
colours = {'United States of America':'#1E90FF',#blue
           'Kingdom of the Netherlands': '#FFA500',#orange
          }

def get_colour(person):
    country = country_by_person[person] if person in country_by_person else ''
    if country in colours:
        return colours[country]
    return '#9370DB' # purple


In [6]:
import networkx as nx


In [8]:
G = nx.Graph()
for _,row in df_graph.iterrows():
    G.add_edge(row['person1'],row['person2'],weight=row['weight'])

In [25]:
from pyvis.network import Network
import numpy as np
def prepare_network(G1):
    # Remove components that are too small (for visualisation purposes)
    for component in list(nx.connected_components(G1)):
        if len(component)<3:
            for node in component:
                G1.remove_node(node)
    # Calculate node degrees
    node_degrees = dict(G1.degree())
    
    net = Network(notebook=True,cdn_resources='remote',bgcolor='#f5f5f5',height='800px',width='800px')
    
    # Define a scaling factor for font size (adjust as needed)
    font_scale = 5
    node_scale = 5#0

    # Add nodes with sizes and font sizes proportional to their degrees
    for node_id, degree in node_degrees.items():
        node_size = np.log(degree)*node_scale#np.log2(degree) * node_scale # Scale node size (adjust as needed)
        font_size = np.sqrt(degree)*font_scale#degree*font_scale#np.log(degree)*font_scale# * font_scale  # Scale font size
        colour = get_colour(node_id)
        net.add_node(node_id, size=node_size, label=str(node_id),color=colour,
                     shape='dot',
                     font={'size': font_size,'color': 'black'},#colour},
                    labelHighlightBold=True)

    # Add edges with weights
    for u, v, data in G.edges(data=True):
        weight = data['weight']  # Extract the weight of the edge
        net.add_edge(u, v, value=weight,
                     color='#d6ebc3')#average_hex_colors(get_colour(u),get_colour(v)))  # Use 'value' to set the edge weight in Pyvis
        
    # Configure physics for repulsion
        
    net.repulsion(
        node_distance=300,    # Distance between nodes
        central_gravity=5.55, # Strength of the central gravity
        spring_length=150,    # Length of the springs
        spring_strength=0.05, # Strength of the springs
        damping=0.6#,          # Damping factor to stabilize the layout
#         avoid_overlap=1
    )

    net.toggle_physics(True)
    
    return net

In [26]:
net = prepare_network(G)
net.show_buttons(filter_=["physics"])
net.show('graph-wikidata-final-small.html')
# net.write_graphml('valkema.graphml')


graph-wikidata-final-small.html
